# Cluster Congress list of Twitter trolls and bots

First, get features.  We have id and handle for each person on the list.  The id is one of two types: early user ids, which were just numbers, and later user ids, which contain (timestamp, machine id, sequence number). 
* Not sure yet what the boundary line is.  Guess it's anything bigger than 22 bits wide? 

This is the list released (as a pdf) by Congress. Am not the only person looking at this: see e.g 
* https://arxiv.org/pdf/1801.09288.pdf

Stuff abandoned:
* print('{}'.format(list(set(df['user id'].tolist()) ^ set(df2['id'].tolist()))))

Fixits
* Look at machine id: does it mean anything, is it useful?  See notes in https://jbum.com/papers/TrackingTwittersGrowthAfterSnowflake.pdf

In [1]:
import pandas as pd
import math
import datetime
pd.set_option("display.max_rows", 3000)


# Extract everything I can from the userid
def decode_twitter_id(twitterid, verbose=False):
    
    # Weed out old twitter ids
    # put boundary at pow(10,12)
    if twitterid < math.pow(10,12):
        return(twitterid, 0, 0, 0, 0)
    
    twitter_epoch = 1288834974657
    bins = '{0:b}'.format(twitterid)
    if verbose:
        print('{0}\n{1}, {2}, {3}'.format(bins, bins[:-22], bins[-22:-12], bins[-12:]))
        print('{0}, {1}, {2}'.format(int(bins[:-22], base=2), int(bins[-22:-12], base=2), 
                                            int(bins[-12:], base=2)))
    seq = int(bins[-12:], base=2)
    mid = int(bins[-22:-12], base=2)
    tepoch = twitter_epoch + int(bins[:-22], base=2)
    tmillis = tepoch % 1e3
    tstamp = datetime.datetime.utcfromtimestamp(tepoch/1e3).strftime('%Y-%m-%d %H:%M:%S')
    return(seq, mid, tepoch, tstamp, tmillis)


dfusers = pd.read_csv('../other_datasets/congress/exhibit_b_russialinkedtwitterbots.output.csv')
dfusers = dfusers.sort_values('user id')
dfusers = dfusers.reset_index()
dfusers['unpacked'] = dfusers['user id'].apply(decode_twitter_id)
dfusers[['seq', 'mid', 'tepoch', 'tstamp', 'tmillis']] = dfusers['unpacked'].apply(pd.Series)
print('{} users'.format(len(dfusers)))
dfusers

2752 users


,index,user id,handle,unpacked,seq,mid,tepoch,tstamp,tmillis
0,2199,18710816,RobbyDelaware,"(18710816, 0, 0, 0, 0)",18710816,0,0,0,0.0
1,203,34976398,andrey_lischina,"(34976398, 0, 0, 0, 0)",34976398,0,0,0,0.0
2,1530,38875764,Leecory,"(38875764, 0, 0, 0, 0)",38875764,0,0,0,0.0
3,2224,79546124,rose4walker,"(79546124, 0, 0, 0, 0)",79546124,0,0,0,0.0
4,450,79626053,BrianFirst001,"(79626053, 0, 0, 0, 0)",79626053,0,0,0,0.0
5,942,79656273,Fuad_Ibragim0v,"(79656273, 0, 0, 0, 0)",79656273,0,0,0,0.0
6,381,79664972,BettyRoberts212,"(79664972, 0, 0, 0, 0)",79664972,0,0,0,0.0
7,1706,79674957,Mary_Moss090,"(79674957, 0, 0, 0, 0)",79674957,0,0,0,0.0
8,3,79793901,1DRussianFNDM,"(79793901, 0, 0, 0, 0)",79793901,0,0,0,0.0
9,2412,79932154,sqaxwer,"(79932154, 0, 0, 0, 0)",79932154,0,0,0,0.0


In [3]:
# Try steps of 1, 10, 100, 1000 etc to see what's common. Nothing til 1000...
last = 0
for index, row in dfusers.iterrows():
    if (row['user id'] - last < 1000):
        print('{}'.format(last))
    last = row['user id']

1649486916
1651692980
1654391976
1661165312
1671234620
1690487623
1692919442
1699979024
1853237414
2527977731
2577151334
2577817587
2579342573
2582408195
2592432748
2627660745
2730955150
2753138169
3084865707
3085183996
3088961943
3091268752
3092323815
3092507884
4484289856


In [19]:
dfusers['mid'].value_counts()

0      2229
376      45
365      43
352      42
374      37
363      32
377      27
375      26
379      25
378      24
373      22
369      16
353      13
354      12
346      10
360       9
332       9
322       9
342       8
349       8
359       8
367       8
333       7
320       7
323       6
326       6
364       5
356       5
358       5
370       5
366       5
372       5
380       5
348       4
324       3
334       3
336       3
321       3
368       2
382       2
355       2
331       1
337       1
339       1
330       1
347       1
371       1
381       1
Name: mid, dtype: int64

## Look at more detailed (but smaller) list of trolls

In [4]:
datadir = '../drivedatasets/nbc_congress_trolls/'
dfdetailed = pd.read_csv('{}/users.csv'.format(datadir))
dfdetailed

,id,location,name,followers_count,statuses_count,time_zone,verified,lang,screen_name,description,created_at,favourites_count,friends_count,listed_count
0,1.003451e+08,still ⬆️Block⤵️Corner⬇️street,#Ezekiel2517✨...,1053.0,31858.0,NaN,False,en,SCOTTGOHARD,CELEBRITY TRAINER ✨#424W147th✨ #CrossfitCoach ...,Tue Dec 29 23:15:22 +0000 2009,2774.0,1055.0,35.0
1,2.471657e+08,"Chicago, IL",B E C K S T E R✨,650.0,6742.0,Mountain Time (US & Canada),False,en,Beckster319,Rebecca Lynn Hirschfeld Actress.Model.Writer.A...,Fri Feb 04 06:38:45 +0000 2011,7273.0,896.0,30.0
2,2.495389e+08,NaN,Chris Osborne,44.0,843.0,NaN,False,en,skatewake1994,NaN,Wed Feb 09 07:38:44 +0000 2011,227.0,154.0,1.0
3,4.496897e+08,NaN,Рамзан Кадыров,94773.0,10877.0,Moscow,False,ru,KadirovRussia,"Пародийный аккаунт. Озвучиваю то, что политика...",Thu Dec 29 11:31:09 +0000 2011,0.0,7.0,691.0
4,4.718689e+08,"Санкт-Петербург, Россия",Маргарита Саваж,23305.0,18401.0,Volgograd,False,ru,MargoSavazh,Честь имею. Нагло врать не умею.,Mon Jan 23 10:26:25 +0000 2012,1480.0,8021.0,231.0
5,1.039581e+09,Amerika,Dark Nally,22.0,22603.0,Jakarta,False,id,darknally,uptodate news!!!!,Thu Dec 27 13:06:58 +0000 2012,0.0,40.0,16.0
6,1.510489e+09,USA,Laila Heats,3080.0,2228.0,Eastern Time (US & Canada),False,en,lagonehoe,How can you judge me when you don’t know me?,Wed Jun 12 13:07:49 +0000 2013,3504.0,2369.0,12.0
7,1.513801e+09,USA,CC Jack,2760.0,2100.0,Eastern Time (US & Canada),False,en,YouJustCtrlC,ProBlack | Stoner | even jesus was stoned befo...,Thu Jun 13 16:07:23 +0000 2013,2744.0,2700.0,10.0
8,1.517679e+09,"Philadelphia, PA",Mr Moran,879.0,2044.0,Eastern Time (US & Canada),False,en,MrMoraan,I'm getting too cool for this social media shi...,Fri Jun 14 21:58:48 +0000 2013,2981.0,758.0,15.0
9,1.518857e+09,USA,Jonathon Norwood,789.0,2002.0,Eastern Time (US & Canada),False,en,NoJonathonNo,Strive not to be a success but rather to be of...,Sat Jun 15 08:47:02 +0000 2013,2170.0,440.0,8.0


In [18]:
# Look at the common user ids and usernames across the two lists
print('{} common ids'.format(len(set(dfusers['user id'].tolist()) & set(dfdetailed['id'].tolist()))))
print('{} common ids'.format(len(set(dfusers['handle'].str.lower().tolist()) & set(dfdetailed['name'].str.lower().tolist()))))      
dfboth = pd.merge(dfusers[['user id', 'handle']], dfdetailed[['id', 'screen_name', 'name']], left_on='user id', right_on='id')
print('{}'.format(dfboth[dfboth['handle'] != dfboth['screen_name']]))

383 common ids
3 common ids
        user id          handle            id  screen_name name
337  4224912857  southlonestar2  4.224913e+09  RealTEN_GOP  NaN
